In [0]:
%run /Users/pnl0rc8b@emea.royalahold.net/FINALIZED_Notebooks/Functions_Finalized

In [0]:
%run /Users/pnl0rc8b@emea.royalahold.net/FINALIZED_Notebooks/Functions_Tables

#Configure Parameters

In [0]:
ATTRIBUUT = "ndATypeOfGrain" # Selecteer Attribuut
#ATTRIBUUT_new = "ndATypeOfFruit"
Classes_threshold = 11
Overig_threshold = 200 # Class beneden deze threshold, zal worden toegevoegd aan class: 'overig'

Multi_label = False
numerieke_features = load_numerical_features(ATTRIBUUT)

# Fix empty rows

In [0]:
nd_controle = koppel_ndAttributen_new_products(NASA_CONTROLE)
df_geformateerd, multi_label_flag,classes_available = create_overige_class(nd_controle, ATTRIBUUT,Classes_threshold, Overig_threshold)

DF_merged = koppel_preprocessed_text(df_geformateerd, DF_NLP_spark)
DF_merged = fix_empty_fields(DF_merged, ATTRIBUUT)
DF_merged = Correlation_matrix(DF_merged, ATTRIBUUT, show_nan_plot=False, show_correlation_matrix=False)

index_empty, TARGETS = get_targets(DF_merged, ATTRIBUUT)
numerical, text, target_unused, vocab_size_unused = single_prep_controle(numerieke_features, DF_merged, ATTRIBUUT)

# Load models from database, ndTypeOfGrain exists out of multiple models.

In [0]:
from tensorflow import keras
all_model_files = dbutils.fs.ls("dbfs:/FileStore/nd_MODELS/"+str(ATTRIBUUT))
#print(all_model_files)
controle_tabel = DF_merged[['gtin','gln','Lemmatized',ATTRIBUUT]]
total_pred = []
total_test = []
class_names = []

for index, model in enumerate(all_model_files): # loop through all models
    print("\n", model.path)
    dbutils.fs.cp(model.path, "file:/FileStore/nd_MODELS/"+str(ATTRIBUUT)+"/"+model.name)
    model = keras.models.load_model("/FileStore/nd_MODELS/"+str(ATTRIBUUT)+"/"+model.name)

    # TARGET
    target = TARGETS[index]
    print(target)
    
    y_target = DF_merged[str(target)].tolist()
    y_target_tensor = torch.FloatTensor(y_target)
    y_target = y_target_tensor.numpy()
    print(numerical.shape, text.shape, y_target.shape)

    # Predictions
    list_ytest, list_yhat, yhat2 = predictions_2inputs(model, numerical, text, y_target)
    
    controle_tabel['TRUE '+str(target)] = list_ytest
    controle_tabel['PRED '+str(target)] = list_yhat
    total_test.append(list_ytest)
    total_pred.append(list_yhat)
    class_names.append(target)

# Controle Tabellen

In [0]:
controle_tabel = predictions_to_right_format(total_test,total_pred, ATTRIBUUT, index_empty, controle_tabel, DF_merged)
equal, unequal = unequal_and_equal_table(controle_tabel, ATTRIBUUT) # Both spark dataframes

In [0]:
display(unequal)

In [0]:
display(equal)